In [39]:
# quick_eda_mitsui.py
import os, re, numpy as np, pandas as pd
from scipy.stats import rankdata

In [40]:
# pd.set_option("display.width", 160)
# pd.set_option("display.max_columns", 200)

DATA_DIR = "data"

# --- загрузка ---
paths = {
    "train": os.path.join(DATA_DIR, "train.csv"),
    "train_labels": os.path.join(DATA_DIR, "train_labels.csv"),
    "test": os.path.join(DATA_DIR, "test.csv"),
    "target_pairs": os.path.join(DATA_DIR, "target_pairs.csv"),
}
lag_paths = {
    int(re.search(r"(\d+)", f).group(1)): os.path.join(DATA_DIR, "lagged_test_labels", f)
    for f in sorted(os.listdir(os.path.join(DATA_DIR, "lagged_test_labels")))
    if f.endswith(".csv")
}

dfs = {k: pd.read_csv(p) for k, p in paths.items()}
lags = {k: pd.read_csv(p) for k, p in lag_paths.items()}

train, train_labels, test = dfs["train"], dfs["train_labels"], dfs["test"]
target_pairs = dfs["target_pairs"]

print("Shapes:")
print({k: v.shape for k, v in dfs.items()} | {f"lag_{k}": v.shape for k, v in lags.items()})
print()

Shapes:
{'train': (1917, 558), 'train_labels': (1917, 425), 'test': (90, 559), 'target_pairs': (424, 3), 'lag_1': (90, 108), 'lag_2': (90, 108), 'lag_3': (90, 108), 'lag_4': (90, 108)}



## Lags

In [31]:
lags[1]

,date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_97,target_98,target_99,target_100,target_101,target_102,target_103,target_104,target_105,label_date_id
0,1829,NaN,NaN,0.017868,-0.000205,-0.016391,-0.013827,0.009972,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000065,NaN,NaN,NaN,NaN,1827
1,1830,0.002560,-0.004592,-0.001776,0.000271,-0.016696,-0.020025,0.002514,0.002204,-0.011962,...,0.021837,0.010965,0.026344,-0.019349,0.013369,-0.002225,0.005445,-0.020425,0.011988,1828
2,1831,0.005346,-0.014539,0.019542,0.014626,-0.011631,-0.009223,-0.005199,-0.026092,-0.003865,...,-0.014079,-0.004723,0.017432,0.012572,-0.010911,-0.016023,0.005366,-0.007975,-0.001883,1829
3,1832,0.000082,-0.005226,0.011452,0.013346,0.008228,-0.014819,-0.011792,-0.007148,0.005712,...,-0.006488,0.018374,0.005335,-0.013671,0.002962,-0.008754,-0.006464,0.019236,0.001574,1830
4,1833,-0.011469,0.016613,-0.023765,-0.018744,-0.011878,0.007257,0.019829,0.006618,-0.015351,...,0.093178,-0.033507,0.094619,-0.010075,0.004967,-0.002408,0.029927,-0.010447,0.009136,1831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1914,-0.012596,0.009309,0.007947,0.006477,-0.026216,-0.029487,-0.003287,0.011751,-0.027610,...,0.011109,0.013837,0.044612,-0.008105,0.021655,0.000476,-0.019934,0.001007,0.006068,1912
86,1915,-0.000736,-0.001292,-0.005137,-0.007961,-0.008494,-0.011547,0.004263,0.005670,-0.004822,...,0.001084,0.003211,0.006946,0.000667,0.016832,-0.002081,0.000489,-0.010563,-0.005520,1913
87,1916,-0.002294,0.012898,0.009978,0.001567,0.002596,-0.007373,0.007554,0.002661,0.004083,...,-0.010641,0.010877,-0.018388,0.017963,-0.004550,-0.001402,-0.016843,0.023616,0.010070,1914
88,1917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.004368,NaN,NaN,NaN,NaN,1915


In [32]:
lags[2]

,date_id,target_106,target_107,target_108,target_109,target_110,target_111,target_112,target_113,target_114,...,target_203,target_204,target_205,target_206,target_207,target_208,target_209,target_210,target_211,label_date_id
0,1830,NaN,NaN,-0.012551,-0.033087,0.033786,0.033152,0.017060,0.006372,0.015137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011162,1827
1,1831,0.004287,-0.027723,-0.012212,-0.028327,0.014351,0.043223,0.025457,0.013672,0.002363,...,-0.043418,-0.018863,0.014931,0.008467,0.018579,-0.048603,0.034468,-0.001175,0.009585,1828
2,1832,0.011360,-0.036678,-0.010981,-0.003402,-0.003930,0.031374,0.000380,-0.024238,-0.000167,...,-0.050206,-0.041814,0.019121,0.030084,0.002206,-0.029841,0.018958,0.013750,0.023990,1829
3,1833,-0.000987,-0.001651,-0.002638,-0.003649,0.012960,-0.001749,0.010564,-0.002772,0.010476,...,0.025865,-0.014746,-0.004260,0.006637,-0.004168,-0.011127,0.048700,-0.001111,0.004510,1830
4,1834,-0.004759,-0.013984,-0.017194,-0.017254,0.022654,0.009717,0.022626,0.023917,-0.002657,...,0.024728,-0.006170,-0.000292,-0.034359,-0.004735,-0.010595,0.051959,0.000821,-0.006448,1831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1915,-0.006293,0.001485,0.000508,-0.034710,0.042518,0.033226,0.030417,0.034371,0.039288,...,0.007572,0.056727,0.003766,-0.012843,0.053636,-0.023721,0.013082,-0.042770,0.009791,1912
86,1916,0.001649,0.001502,-0.005424,-0.005898,0.025314,-0.000496,-0.005337,0.001970,0.018270,...,0.000585,0.031050,0.004194,-0.015050,0.009857,-0.007490,-0.017650,-0.019707,-0.013996,1913
87,1917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002695,-0.004560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1914
88,1918,0.009691,-0.008211,0.004246,0.013552,-0.027762,-0.006342,-0.003457,-0.002639,-0.020392,...,-0.021892,-0.018229,-0.014328,-0.009981,-0.023334,0.004415,-0.018531,0.032968,0.006244,1915


In [33]:
lags[3]

,date_id,target_212,target_213,target_214,target_215,target_216,target_217,target_218,target_219,target_220,...,target_309,target_310,target_311,target_312,target_313,target_314,target_315,target_316,target_317,label_date_id
0,1831,0.000778,-0.019080,-0.023775,-0.037431,-0.028383,0.044718,-0.021978,-0.041951,NaN,...,NaN,0.003093,NaN,0.017557,NaN,NaN,NaN,NaN,NaN,1827
1,1832,0.005852,-0.022221,-0.019123,-0.034575,-0.015824,0.020098,-0.013765,-0.040860,-0.000051,...,-0.015164,0.021710,0.030307,0.019206,0.005672,-2.548676e-02,0.030122,0.022183,0.031571,1828
2,1833,-0.013472,0.004279,-0.017279,-0.012442,-0.007557,0.015280,-0.012066,-0.020875,-0.013257,...,-0.014179,0.003626,0.032447,0.005762,0.016799,-6.278727e-07,0.008909,0.023005,0.077329,1829
3,1834,-0.008813,0.002121,-0.016291,0.003913,-0.024127,0.009025,-0.018747,-0.024458,-0.021266,...,-0.013230,-0.007912,0.027885,-0.006919,0.024987,-2.038526e-02,0.018940,-0.020644,0.079341,1830
4,1835,-0.007298,0.018325,-0.017851,0.015855,-0.006896,0.003347,-0.026891,-0.007556,-0.010219,...,0.007587,-0.033888,0.018132,-0.005430,0.014611,-1.908291e-02,0.033735,-0.049945,0.072558,1831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1916,0.005490,-0.000202,-0.019410,-0.023584,-0.048324,0.032114,-0.008614,-0.017391,-0.004463,...,0.004779,-0.046495,0.047107,0.035988,-0.020740,-1.389059e-02,0.026057,-0.015948,0.041345,1912
86,1917,0.007308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.017009,NaN,0.007336,NaN,NaN,NaN,NaN,NaN,1913
87,1918,0.003184,0.004654,0.014464,0.012247,0.021956,-0.016149,-0.004876,-0.002373,0.003997,...,0.003427,0.012532,0.015830,-0.003069,0.011357,-3.676607e-03,-0.006195,-0.013087,-0.050104,1914
88,1919,-0.007805,0.003527,-0.003270,-0.002854,0.013003,-0.003452,-0.001103,0.006186,-0.001834,...,-0.004787,0.008324,0.011420,-0.015612,0.009330,1.017366e-02,-0.009512,-0.022548,-0.038232,1915


In [34]:
lags[4]

,date_id,target_318,target_319,target_320,target_321,target_322,target_323,target_324,target_325,target_326,...,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423,label_date_id
0,1832,0.001082,-0.020023,0.064526,0.028037,0.004506,-0.023554,-0.029856,NaN,NaN,...,0.019701,NaN,-0.027030,0.043602,0.027982,NaN,NaN,0.002177,NaN,1827
1,1833,-0.001296,-0.015281,0.041474,0.009498,-0.005352,0.009397,-0.027311,0.046440,-0.009329,...,0.012081,-0.020068,0.002858,0.019154,0.019018,0.003875,-0.035202,0.011246,0.099241,1828
2,1834,-0.002375,-0.021250,0.041091,0.020435,-0.007740,0.010090,-0.018724,0.033513,0.018609,...,0.016166,-0.028919,-0.007297,0.033262,0.023174,-0.028512,-0.017900,-0.002096,0.121451,1829
3,1835,-0.001846,0.000320,0.016153,0.021035,-0.007409,0.023807,-0.008369,0.045870,0.010528,...,-0.007742,-0.018436,0.004691,0.013311,0.000589,-0.014500,-0.046444,0.009058,0.109246,1830
4,1836,-0.000929,-0.004576,0.004014,0.006592,-0.020097,0.030620,-0.009754,0.050258,0.017386,...,-0.018850,-0.025373,0.031197,0.005873,-0.005650,-0.022926,-0.027990,0.011267,0.091318,1831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1917,0.006182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.042052,NaN,NaN,NaN,NaN,0.031006,NaN,1912
86,1918,0.001901,0.003547,-0.006838,0.000816,-0.000361,-0.000778,0.002448,-0.034985,0.038066,...,-0.003349,0.010912,0.006198,-0.009216,-0.003038,-0.026082,-0.008057,-0.002069,-0.141053,1913
87,1919,0.002587,0.002489,0.002929,0.008977,-0.001248,-0.000441,0.007197,0.004428,0.017960,...,0.004044,0.004459,0.002619,0.001308,-0.006772,-0.019918,-0.013304,-0.005527,-0.127688,1914
88,1920,-0.002140,-0.003151,0.006951,0.004380,0.008872,-0.009783,-0.011986,-0.012078,0.016320,...,0.012842,0.009076,0.000932,0.011613,0.003825,0.024350,-0.006928,0.006805,-0.012187,1915


## Dataframes

In [20]:
dfs["train"].head(2)

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.11963,0.078135,13.822740,0.059163
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.12052,0.079066,13.888146,0.059895


In [36]:
train

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
0,0,2264.5,7205.0,2570.0,3349.0,NaN,NaN,NaN,NaN,NaN,...,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.119630,0.078135,13.822740,0.059163
1,1,2228.0,7147.0,2579.0,3327.0,NaN,NaN,NaN,NaN,NaN,...,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.120520,0.079066,13.888146,0.059895
2,2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,...,1.692724,0.780186,0.894004,0.697490,0.067394,0.091478,0.120809,0.079287,13.983675,0.060037
3,3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,...,1.683111,0.785329,0.889439,0.698502,0.067639,0.091558,0.121021,0.079285,14.035571,0.059983
4,4,2175.0,7125.0,2604.0,3386.0,NaN,NaN,NaN,NaN,NaN,...,1.684816,0.787264,0.891042,0.701485,0.067443,0.091266,0.121055,0.078925,14.013760,0.059503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,...,1.864661,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388
1913,1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,...,1.863539,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630
1914,1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,...,1.860067,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457
1915,1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,...,1.859624,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368


In [67]:
dfs["train_labels"].head()

,date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,...,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
0,0,0.005948,-0.002851,-0.004675,-0.000639,NaN,NaN,-0.006729,0.006066,NaN,...,NaN,0.021239,-0.005595,NaN,-0.004628,0.033793,NaN,0.038234,NaN,0.027310
1,1,0.005783,-0.024118,-0.007052,-0.018955,-0.031852,-0.019452,0.003002,-0.006876,-0.002042,...,0.003377,0.021372,-0.001517,0.012846,0.010547,0.030527,-0.000764,0.025021,0.003548,0.020940
2,2,0.001048,0.023836,-0.008934,-0.022060,NaN,NaN,0.037449,0.007658,NaN,...,-0.006712,0.009308,0.001857,-0.012761,-0.002345,0.017529,-0.005394,0.004835,-0.009075,0.001706
3,3,0.001700,-0.024618,0.011943,0.004778,NaN,NaN,-0.012519,-0.016896,NaN,...,NaN,0.036880,-0.015189,NaN,0.008118,0.001079,NaN,-0.015102,NaN,-0.033010
4,4,-0.003272,0.005234,0.006856,0.013312,0.023953,0.010681,-0.011649,0.002019,0.003897,...,NaN,0.004937,NaN,-0.006673,-0.016105,-0.004885,NaN,NaN,0.009514,NaN


In [38]:
dfs["test"].head(2)

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP,is_scored
0,1827,2684.5,9190.0,1967.0,2942.0,13623.0,13920.0,13618.0,4696.0,4692.0,...,0.631633,0.808485,0.510666,0.051733,0.071654,0.079797,0.048828,13.631347,0.043845,True
1,1828,2691.5,9275.0,1985.0,2963.5,13640.0,13922.0,13634.0,4613.0,4613.0,...,0.633526,0.812571,0.514785,0.051802,0.071793,0.080214,0.048912,13.743387,0.043778,True


In [48]:
dfs["target_pairs"]["lag"].value_counts()

lag
1    106
2    106
3    106
4    106
Name: count, dtype: int64

In [51]:
set(dfs["target_pairs"]["pair"])

{'FX_AUDCAD - JPX_Platinum_Standard_Futures_Close',
 'FX_AUDCHF - JPX_Platinum_Standard_Futures_Close',
 'FX_AUDJPY - LME_PB_Close',
 'FX_AUDJPY - LME_ZS_Close',
 'FX_AUDUSD - JPX_Platinum_Standard_Futures_Close',
 'FX_CADCHF - JPX_Platinum_Standard_Futures_Close',
 'FX_CADCHF - LME_CA_Close',
 'FX_CADJPY - JPX_Platinum_Standard_Futures_Close',
 'FX_CADJPY - LME_AH_Close',
 'FX_CADJPY - LME_PB_Close',
 'FX_CADUSD - LME_AH_Close',
 'FX_CADUSD - LME_CA_Close',
 'FX_CHFJPY - LME_AH_Close',
 'FX_CHFJPY - LME_ZS_Close',
 'FX_EURAUD - LME_AH_Close',
 'FX_EURAUD - LME_CA_Close',
 'FX_EURAUD - LME_PB_Close',
 'FX_EURCAD - LME_AH_Close',
 'FX_EURCAD - LME_ZS_Close',
 'FX_EURCHF - LME_CA_Close',
 'FX_EURGBP - JPX_Gold_Standard_Futures_Close',
 'FX_EURGBP - JPX_Platinum_Standard_Futures_Close',
 'FX_EURJPY - JPX_Gold_Standard_Futures_Close',
 'FX_EURJPY - JPX_Platinum_Standard_Futures_Close',
 'FX_EURJPY - LME_ZS_Close',
 'FX_EURNZD - JPX_Gold_Standard_Futures_Close',
 'FX_EURNZD - LME_PB_Close',

In [65]:
test

,date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,...,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP,is_scored
0,1827,2684.5,9190.0,1967.0,2942.0,13623.0,13920.0,13618.0,4696.0,4692.0,...,0.631633,0.808485,0.510666,0.051733,0.071654,0.079797,0.048828,13.631347,0.043845,True
1,1828,2691.5,9275.0,1985.0,2963.5,13640.0,13922.0,13634.0,4613.0,4613.0,...,0.633526,0.812571,0.514785,0.051802,0.071793,0.080214,0.048912,13.743387,0.043778,True
2,1829,2646.0,9284.5,1971.0,2914.0,13634.0,13923.0,13638.0,4647.0,4632.0,...,0.632156,0.811948,0.513278,0.051902,0.071630,0.080134,0.048971,13.766241,0.043774,True
3,1830,2634.0,9223.5,1967.0,2900.0,13681.5,13962.0,13680.0,4630.0,4631.0,...,0.629661,0.815155,0.513271,0.051907,0.071972,0.080325,0.048968,13.864629,0.043876,True
4,1831,2623.5,9232.0,1949.0,2846.5,13849.5,14141.0,13844.0,4699.5,4703.0,...,0.630969,0.816284,0.515051,0.051867,0.071790,0.080475,0.049027,13.847691,0.043736,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,1912,2450.0,9523.5,1961.5,2676.5,15086.0,15440.0,15085.0,4461.5,4467.0,...,0.598318,0.827529,0.495125,0.049224,0.072574,0.080968,0.046175,14.058107,0.041388,True
86,1913,2471.5,9519.5,1980.5,2710.5,15165.0,15509.0,15162.0,4495.0,4490.0,...,0.594400,0.824390,0.490018,0.049409,0.072828,0.080671,0.046113,14.082236,0.041630,True
87,1914,2471.5,9533.5,1974.0,2693.0,15040.0,15477.0,15044.0,4544.5,4555.0,...,0.595250,0.822392,0.489529,0.049095,0.073232,0.081083,0.045901,14.126606,0.041457,True
88,1915,2456.0,9500.5,1970.0,2697.5,15420.0,15752.0,15420.0,4670.0,4685.0,...,0.597780,0.817224,0.488520,0.049205,0.073018,0.081170,0.045987,14.095322,0.041368,True


## Further look-into

In [5]:
# --- определение id/даты и таргетов ---
def find_time_id_col(df):
    for c in df.columns:
        if re.search(r"(date|time).*id|^date$|^time$", c, re.I):
            return c
    # fallback: первая колонка, если числовая/монотонная
    c0 = df.columns[0]
    return c0

time_col = find_time_id_col(train)
print(f"time/id column: {time_col}")

time/id column: date_id


In [6]:
# таргеты берём из train_labels: все кроме time_col
target_cols = [c for c in train_labels.columns if c != time_col]
print(f"#targets: {len(target_cols)}")
print("sample targets:", target_cols[:10])
print()

#targets: 424
sample targets: ['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']



In [7]:
# --- приведение типов времени (по возможности) ---
def coerce_time(s):
    if np.issubdtype(s.dtype, np.integer):
        return s  # индекс-дни
    try:
        return pd.to_datetime(s)
    except Exception:
        return s  # оставим как есть

for df in [train, train_labels, test, *lags.values()]:
    if time_col in df.columns:
        df[time_col] = coerce_time(df[time_col])

# --- базовая инфа по фичам ---
feature_cols = [c for c in train.columns if c != time_col]
print(f"#features train: {len(feature_cols)}; #features test: {len([c for c in test.columns if c != time_col])}")
print("feature sample:", feature_cols[:10])
print()

#features train: 557; #features test: 558
feature sample: ['LME_AH_Close', 'LME_CA_Close', 'LME_PB_Close', 'LME_ZS_Close', 'JPX_Gold_Mini_Futures_Open', 'JPX_Gold_Rolling-Spot_Futures_Open', 'JPX_Gold_Standard_Futures_Open', 'JPX_Platinum_Mini_Futures_Open', 'JPX_Platinum_Standard_Futures_Open', 'JPX_RSS3_Rubber_Futures_Open']



In [8]:
# --- пропуски ---
def top_missing(df, k=20):
    m = df.isna().mean().sort_values(ascending=False)
    m = m[m>0]
    return m.head(k)

print("Top missing in train:")
print(top_missing(train))
print("\nTop missing in train_labels:")
print(top_missing(train_labels))
print()

Top missing in train:
US_Stock_GOLD_adj_volume                          0.870631
US_Stock_GOLD_adj_low                             0.870631
US_Stock_GOLD_adj_high                            0.870631
US_Stock_GOLD_adj_open                            0.870631
US_Stock_GOLD_adj_close                           0.870631
JPX_Platinum_Mini_Futures_settlement_price        0.059990
JPX_Gold_Mini_Futures_settlement_price            0.059990
JPX_Platinum_Standard_Futures_Close               0.059990
JPX_RSS3_Rubber_Futures_Close                     0.059990
JPX_Gold_Mini_Futures_Volume                      0.059990
JPX_Gold_Standard_Futures_Volume                  0.059990
JPX_Platinum_Mini_Futures_Volume                  0.059990
JPX_Platinum_Standard_Futures_Volume              0.059990
JPX_RSS3_Rubber_Futures_Volume                    0.059990
JPX_Gold_Mini_Futures_open_interest               0.059990
JPX_Gold_Rolling-Spot_Futures_settlement_price    0.059990
JPX_RSS3_Rubber_Futures_settlement

In [12]:
# --- быстрые статистики по таргетам ---
desc_targets = train_labels[target_cols].agg(["mean","std","skew","min","max"]).T.sort_values("std", ascending=False)
print("Targets stats (top by std):")
print(desc_targets.head(10))
print()

Targets stats (top by std):
                mean       std      skew       min       max
target_411 -0.000190  0.075032  3.226601 -0.383897  0.908524
target_393 -0.000036  0.073611  0.002115 -0.428705  0.388126
target_335  0.002357  0.072846 -4.708521 -0.901004  0.513583
target_404  0.003035  0.072375 -0.119486 -0.314347  0.285212
target_329  0.001651  0.071203 -0.005600 -0.269028  0.290101
target_352 -0.001188  0.070859 -3.541074 -0.925517  0.472875
target_343  0.000193  0.068704 -1.522091 -0.636494  0.331226
target_357  0.001054  0.067308  1.854363 -0.380032  0.687439
target_235 -0.001565  0.063576  4.825546 -0.431697  0.896104
target_224 -0.000334  0.063458 -3.200608 -0.915148  0.433969



In [13]:
# --- быстрые корреляции таргетов между собой (сэмпл для скорости) ---
sample_tgts = target_cols[:50]  # ограничим
corr_sample = train_labels[sample_tgts].corr(method="spearman")
print("Spearman corr across first 50 targets: shape", corr_sample.shape)
print("Median(|corr|) over sample:", corr_sample.abs().where(~np.eye(len(corr_sample), dtype=bool)).stack().median())
print()

Spearman corr across first 50 targets: shape (50, 50)
Median(|corr|) over sample: 0.18645895059959086



In [14]:
# --- target_pairs просмотр ---
print("target_pairs head:")
print(target_pairs.head())
print("target_pairs columns:", list(target_pairs.columns))
for c in target_pairs.columns:
    if target_pairs[c].dtype == object:
        print(f"unique({c}) sample:", target_pairs[c].dropna().unique()[:5])
print()

target_pairs head:
     target  lag                                            pair
0  target_0    1                           US_Stock_VT_adj_close
1  target_1    1            LME_PB_Close - US_Stock_VT_adj_close
2  target_2    1                     LME_CA_Close - LME_ZS_Close
3  target_3    1                     LME_AH_Close - LME_ZS_Close
4  target_4    1  LME_AH_Close - JPX_Gold_Standard_Futures_Close
target_pairs columns: ['target', 'lag', 'pair']
unique(target) sample: ['target_0' 'target_1' 'target_2' 'target_3' 'target_4']
unique(pair) sample: ['US_Stock_VT_adj_close' 'LME_PB_Close - US_Stock_VT_adj_close'
 'LME_CA_Close - LME_ZS_Close' 'LME_AH_Close - LME_ZS_Close'
 'LME_AH_Close - JPX_Gold_Standard_Futures_Close']



In [15]:
# --- проверка join по времени ---
merged = pd.merge(train[[time_col]], train_labels[[time_col] + target_cols], on=time_col, how="inner")
print("merge(train_time, train_labels) rows:", merged.shape[0])
print()

merge(train_time, train_labels) rows: 1917



In [16]:
# --- пример подготовки train X, y (без утечек) ---
X = train.set_index(time_col).sort_index()
y = train_labels.set_index(time_col).sort_index()[target_cols]
X, y = X.loc[X.index.intersection(y.index)], y.loc[X.index.intersection(y.index)]
print("Aligned X,y shapes:", X.shape, y.shape)
print()

Aligned X,y shapes: (1917, 557) (1917, 424)



In [17]:
# --- функция метрики (Sharpe по Spearman) ---
def spearman_sharpe(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
    assert list(y_true.columns) == list(y_pred.columns)
    cols = y_true.columns
    rhos = []
    for c in cols:
        a = y_true[c].values
        b = y_pred[c].values
        mask = np.isfinite(a) & np.isfinite(b)
        if mask.sum() < 3:
            continue
        ra = rankdata(a[mask])
        rb = rankdata(b[mask])
        # корреляция Пирсона между рангами
        ra = (ra - ra.mean())/ra.std()
        rb = (rb - rb.mean())/rb.std()
        rho = float(np.mean(ra*rb))
        rhos.append(rho)
    rhos = np.array(rhos, dtype=float)
    return float(rhos.mean() / (rhos.std() + 1e-12))

In [18]:
# --- sanity check метрики на шуме ---
rng = np.random.default_rng(0)
y_true_demo = y.iloc[:200, :100]
y_pred_noise = y_true_demo + rng.normal(0, y_true_demo.std(axis=0), size=y_true_demo.shape)
y_pred_rankish = y_true_demo.rank(axis=0)  # заведомо рангово-похожее
print("Metric noisy preds:", spearman_sharpe(y_true_demo, y_pred_noise))
print("Metric rankish preds:", spearman_sharpe(y_true_demo, y_pred_rankish))
print()

print("Done.")

Metric noisy preds: 16.69089169391604
Metric rankish preds: 999935267620.0609

Done.
